In [1]:
import os

In [2]:
%pwd

'c:\\Users\\autom\\OneDrive\\Desktop\\End-to-end-Machine-Learning-Project-with-MLflow-Salary\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\autom\\OneDrive\\Desktop\\End-to-end-Machine-Learning-Project-with-MLflow-Salary'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from MlflowProject.constants import *
from MlflowProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from MlflowProject import logger
from MlflowProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-26 11:18:49,001: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-26 11:18:49,004: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-26 11:18:49,006: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-26 11:18:49,008: INFO: common: created directory at: artifacts]
[2025-05-26 11:18:49,011: INFO: common: created directory at: artifacts/data_ingestion]


[2025-05-26 11:18:50,092: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 472
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "616b87186026baab1c001df186159187d8af859d1d97d4e6769c19e588859b99"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F03A:1F21C:2886A:534CA:683400C3
Accept-Ranges: bytes
Date: Mon, 26 May 2025 05:48:54 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830034-CCU
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1748238534.315791,VS0,VE325
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 14a49526d92b507775c5a68ee6edf1a6a1fc7c84
Expires: Mon, 26 May 2025 05:53:54 GMT
Source-Age: 0

]
